In [1]:
from pathlib import Path
import torch
import pandas as pd
from forest_fire_snn import SNN


BASE_DIR = Path.cwd()

net = SNN()
model_path = BASE_DIR / "trained_snn_model.pth"
net.load_state_dict(torch.load(model_path, map_location="cpu"))
net.eval()
print("Trained model loaded successfully.")

csv_path = BASE_DIR / "firedata_validation.csv"
df = pd.read_csv(csv_path)

# normalize once
df["Temp"] = df["Temp"] / 100
df["Humidity"] = df["Humidity"] / 100
df["CO2"] = df["CO2"] / 5000

num_steps = 35

preds, confs, score0, score1 = [], [], [], []

with torch.no_grad():
    for _, row in df.iterrows():
        x0 = torch.tensor(
            [[row["Temp"], row["Audio"], row["Humidity"], row["CO2"]]],
            dtype=torch.float32
        )
        x = x0.unsqueeze(0).repeat(num_steps, 1, 1)

        spk_rec, mem_rec = net(x)
        spike_counts = spk_rec.sum(dim=0)

        s0 = float(spike_counts[0, 0].item())
        s1 = float(spike_counts[0, 1].item())
        pred = int(torch.argmax(spike_counts, dim=1).item())
        conf = float(spike_counts[0, pred].item() / (spike_counts[0].sum().item() + 1e-9))

        score0.append(s0); score1.append(s1)
        preds.append(pred); confs.append(conf)

df["Spikes_Class0"] = score0
df["Spikes_Class1"] = score1
df["Pred_Fire"] = preds
df["Pred_Conf"] = confs

out_path = BASE_DIR / "firedata_validation_with_preds.csv"
df.to_csv(out_path, index=False)
print("Saved:", out_path)
print(df.head())

Trained model loaded successfully.
Saved: c:\Users\Aishik C\Desktop\neuromorphic\neuromorphic_project\firedata_validation_with_preds.csv
       Temp     Audio  Humidity       CO2  Fire  Spikes_Class0  Spikes_Class1  \
0  0.237190  0.426199  0.733033  0.212341     0            6.0            0.0   
1  0.537243  0.182297  0.054337  0.651254     1            0.0            6.0   
2  0.461913  0.735251  0.206928  0.712489     1            0.0            3.0   
3  0.431333  0.248078  0.518643  0.164737     0            0.0            0.0   
4  0.370759  0.101029  0.456088  0.181836     0            0.0            0.0   

   Pred_Fire  Pred_Conf  
0          0        1.0  
1          1        1.0  
2          1        1.0  
3          0        0.0  
4          0        0.0  


In [2]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_true = df["Fire"].astype(int).to_numpy()
y_pred = df["Pred_Fire"].astype(int).to_numpy()

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, digits=4))

Accuracy: 0.9376666666666666
Confusion matrix:
 [[15108     0]
 [ 1870 13022]]
              precision    recall  f1-score   support

           0     0.8899    1.0000    0.9417     15108
           1     1.0000    0.8744    0.9330     14892

    accuracy                         0.9377     30000
   macro avg     0.9449    0.9372    0.9374     30000
weighted avg     0.9445    0.9377    0.9374     30000



In [3]:
# sanity check balance
print(df["Fire"].value_counts())
print(df["Fire"].value_counts(normalize=True))

# accuracy + confusion breakdown (no sklearn needed)
y_true = df["Fire"].astype(int).to_numpy()
y_pred = df["Pred_Fire"].astype(int).to_numpy()

tp = ((y_true==1) & (y_pred==1)).sum()
tn = ((y_true==0) & (y_pred==0)).sum()
fp = ((y_true==0) & (y_pred==1)).sum()
fn = ((y_true==1) & (y_pred==0)).sum()

acc = (tp+tn)/len(y_true)
precision = tp/(tp+fp+1e-9)
recall = tp/(tp+fn+1e-9)
f1 = 2*precision*recall/(precision+recall+1e-9)

print(f"Accuracy:  {acc:.4f}")
print(f"TP={tp}  TN={tn}  FP={fp}  FN={fn}")
print(f"Precision: {precision:.4f}  (how often 'fire' is correct)")
print(f"Recall:    {recall:.4f}  (how many fires you catch)")
print(f"F1:        {f1:.4f}")


Fire
0    15108
1    14892
Name: count, dtype: int64
Fire
0    0.5036
1    0.4964
Name: proportion, dtype: float64
Accuracy:  0.9377
TP=13022  TN=15108  FP=0  FN=1870
Precision: 1.0000  (how often 'fire' is correct)
Recall:    0.8744  (how many fires you catch)
F1:        0.9330
